In [ ]:
pip install pandas scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
reci = pd.read_csv('E:/더조은/프로젝트/NEW_3/10000recipe Preprocessing.csv')
reci.head()

,Unnamed: 0,레시피일련번호,조리순서,레시피제목,요리명,등록자ID,등록자명,조회수,추천수,스크랩수,요리방법별명,요리상황별명,요리재료별명,요리종류별명,요리소개,요리재료내용,요리인분명,요리난이도명,요리시간명,최초등록일시
0,0,6852249,[조리순서]\n1.밥은 넓은 접시에 펼쳐 뜨거운 김을 날린다.밥 1공기\n2.양념장...,백전백승! 실패없는 김치볶음밥,김치볶음밥,gdubu33,만개의레시피,366065,1324,6914,볶음,일상,쌀,밥/죽/떡,이 레시피대로만 따라하면 절대! 실패따윈 두려워 하지 않아도 되! 세상의 모든 레...,[필수재료] 밥 1공기| 다진 양파 3큰술| 다진 김치 1/2컵| 다진 햄 3큰술 ...,1인분,아무나,15분이내,20160706123205
1,1,6859485,[조리순서]\n1.감자는 손질하여 한입 크기로 썰어 준비한다.감자 2개(360g)감...,매일 먹는 밑반찬 메뉴로 추천!! 감자조림,감자조림,gdubu33,만개의레시피,478527,1102,8221,조림,일상,채소류,밑반찬,만들어두면 마음&배가 든든해지는 달큰 짭잘 감자조림! 세상의 모든 레시피 만개의레시피,[필수재료] 감자 2개(360g)| 물 1cup [양념재료] 간장 5T| 설탕 1T...,2인분,아무나,30분이내,20161031112035
2,2,6846655,[조리순서]\n1.물 한 컵에 돼지고기를 넣고 끓여주세요물 대신 쌀뜨물을 사용하시면...,새마을식당 7분김치찌개 만들기,김치찌개,76918586,임루시,218649,1077,7594,끓이기,일상,채소류,찌개,새마을식당 김치찌개 따라잡기,[재료] 김치| 돼지고기(찌개용)| 양파 1/2개| 파 약간 [양념] 김치국물 2~...,2인분,아무나,15분이내,20160418201815
3,3,6841008,[조리순서]\n1.재료는 먹기 좋은 크기도 썰어서 준비해주세요^^\n2.고추장 2스...,백종원 제육볶음 레시피^^ 백종원의 노하우가 들어있는 손쉬운 레시피 백종원 제육볶음...,제육볶음,only1a2a3a,랑이의보물상자,976200,915,12418,볶음,술안주,돼지고기,메인반찬,백종원의 제육볶음 레시피로 간단하게 밥반찬 또는 야식 술안주 뚝딱 만들어봤습니다^^,[재료] 돼지고기 600g| 양파 1개| 청양고추 2개| 대파 1뿌리 [양념] 설탕...,4인분,아무나,30분이내,20160103095322
4,4,6857726,[조리순서]\n1.고등어는 내장을 제거하시고 핏물이 남지 않도록 깨끗이 씻어 준비합...,비린내 걱정없는 백종원 고등어조림,고등어조림,heeya120608,심술이네집밥이야기,297242,887,10924,조림,일상,해물류,메인반찬,집밥 백선생 시즌 2에서 소개된 비린내 없이 맛있게 만들수 있는 백종원씨의 고등어...,[재료] 고등어 2마리| 무| 양파| 대파| 청양고추 [양념] 설탕 1큰술| 다진마...,4인분,초급,60분이내,20160930162922


In [ ]:
#  텍스트를 n-그램으로 벡터화하기
def ngram_vectorize(texts, n=2):  # n=2 : 연속된 두 문자씩의 조합
    # CountVectorizer 초기화 - 문자 단위로 분석수행.
    vectorizer = CountVectorizer(analyzer='char', ngram_range=(n, n))
    # 텍스트 데이터 벡터화 - n그램 생성, 벡터화
    vectors = vectorizer.fit_transform(texts)
    # 벡터 배열로 변환
    return vectors.toarray(), vectorizer

In [ ]:
# 두벡터간의 코사인 유사도 계산하여 반환하는 함수 정의
def cosine_similarity_score(v1, v2):
    # 두 벡터간의 코사인 유사도 계산(완전히 동일하면 1, 완전히 반대방향이면 -1, 서로 직교하면 0)
    return cosine_similarity([v1], [v2])[0][0]

In [ ]:
# 조회수가 높고 스크랩수가 많은 순으로 3개 추천
def chat(request):
    max_similarity = 0
    best_matches = []

    request_vector, vectorizer = ngram_vectorize([request])
    request_vector = request_vector[0]

    for index, row in reci.iterrows():
        word = row['요리명']
        word_vector = vectorizer.transform([word]).toarray()[0]
        similarity = cosine_similarity_score(request_vector, word_vector)

        # 사용자 입력과 음식명 사이의 유사도가 높을 때만 업데이트
        if similarity > 0.5:
            best_matches.append((similarity,row))

    # 가장 유사한 음식명에 대응하는 응답 텍스트 생성
    if best_matches:
        best_matches.sort(key = lambda x: (x[1]['조회수'], x[1]['스크랩수']), reverse = True)
        responses =[]

        for i, match in enumerate(best_matches[:3], start=1):  # 최대 3개까지 출력
            response = f"\n<{i}. {match[1]['레시피제목']}>\n\n{match[1]['요리재료내용']}\n{match[1]['조리순서']}"
            if i == 1:
                response = f"{match[1]['요리명']} 레시피를 알려드리겠습니다.\n" + response
            responses.append(response)

        return '\n'.join(responses)

    return '음식명으로 검색해주세요^^'

In [ ]:
# 예시 입력
while True:
    req = input("채팅을 입력하세요! (종료하려면 '끝'을 입력하세요): ")
    if req == '끝':
        break
    else:
        print('You : ' , req)
        print("쿠킹파트너 : ", chat(req))

You :  오이소박이
쿠킹파트너 :  오이소박이 레시피를 알려드리겠습니다.

<1. 아삭하게 오이소박이 담그는 법>

[재료] 오이 8개| 천일염 적당량 [양념] 부추 1/2단| 양파 1/2개| 홍고추 2개| 다진 마늘 1큰술| 멸치액젓 3큰술| 새우젓 2-3큰술| 깨 1큰술| 설탕 2큰술| 매실청 1큰술| 고춧가루 7큰술| 다진생강 약간
[조리순서]
1.천일염으로 문질러 세척한 오이는 끝부분을 잘라내고, 길이로 4등분 한후 길이의 2/3지점 까지 열십자로 칼집을 넣어줍니다.
2.칼집을 넣은 오이를 천일염을 살짝 묻혀줍니다. 이때 오이 겉면에 물기가 살짝 있어야 소금이 붙어요.
3.간편하게 천일염에 묻힌 오이를 위생비닐에 넣고 봉하여 1시간 가량 절여 줍니다, 이때 중간에 두어번 비닐의 위 아래 위치를 바꿔줍니다.
4.소금에 절인 오이를 끓는 소금물에 넣고, 물이 바글바글 끓으려 하면 불을 끄고 3분 정도 그대로 방치합니다.
5.뜨거운 상태의 오이를 헹구지 말고, 바로 체를 받쳐 자연스레 식힙니다.
6.분량대로 소를 만들어 버무려주세요. 조금 간을 보시고 모자라시면 소금과 설탕으로 기호에 맞게 맞춰 주세요.
7.식힌 오이는 손으로 꾹 한번 눌러주면 약간의 물기가 나옵니다. 요정도만 수분을 제거해 주셔도 됩니다.
8.미리 버무려둔 오이소박이 소를 오이 사이에 꽉 채워 주시면 됩니다.
9.소를 채운 오이를 용기에 꾹꾹 눌러 담은 후 요즘같이 더울때는 실온에 반나절만 두었다가 냉장고에 보관하시며 드시면 됩니다.

<2. 오이소박이 오이김치 소박이김치>

[재료] 오이 14개| 부추 2/3단| 당근 작은것 1개| 양파 2개 [양념] 고추가루 1/2컵| 새우젓 2큰술| 다진마늘 1큰술| 매실액| 참깨
[조리순서]
1.오이는 굵은 소금으로 문질러 깨끗이 씻어 줍니다.
2.오이는 통째로 길게 칼집을 넣어 주세요.
3.칼집을 넣은 오이에 팔팔 끓인 소금물을 부어 한시간 정도 절여줍니다.절여진 오이는 차가운물에 행구어 물기를 빼주세요.
4.오이가 휘어지면 잘 절여진거에요.
